---
Lundi 18 - 12 - 2023
# P6-DS-OC | Classifier automatiquement des biens de consommation


<span style="color:#FF69B4" font=10>**TODO**</span>


Step 1 - suite : Étude de faisabilité d'un model de catégorisation


5. Tester deux approches pour la feature extraction : bag-of-words et Tf-idf
6. Test Word2Vec (ou Doc2Vec ou Glove ou FastText)
7. Tester Bert avec une approhce word/sentence embedding
8. Tester USE
---


In [12]:
import pandas as pd
import logging
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from src.pipelines.preprocess import DataProcessor
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import gensim




---
## Feature Extration
---

In [13]:
# Initialize the data processor class to preprocess the data
preprocess = DataProcessor()


"2024-01-14 19:08:34,325 - Data-Preprocessing - INFO - ========== Initializing =========="
"2024-01-14 19:08:34,328 - Data-Preprocessing - INFO - Data directory is loaded !!!"
"2024-01-14 19:08:34,328 - Data-Preprocessing - INFO - Outputs directory is loaded !!!"
"2024-01-14 19:08:37,647 - Data-Preprocessing - INFO - Spacy model en_core_web_trf is loaded !!!"


### Read data


In [14]:
data=pd.read_csv("../outputs/data/data_clean_20240114.csv", encoding="utf-8")
data.head()


,uniq_id,description,category,_len_description,text,_len__text
0,55b85ea15a1536d46b7190ad6fff8ce7,Key Features of Elegance Polyester Multicolor ...,Home Furnishing,1420,key elegance polyester multicolor abstract eye...,998
1,7b72c92c2f6c40268628ec5f14c6d590,Specifications of Sathiyas Cotton Bath Towel (...,Baby Care,444,specification sathiyas cotton bath towel bath ...,371
2,64d5d4a258243731dc7bbb1eef49ad74,Key Features of Eurospa Cotton Terry Face Towe...,Baby Care,1258,key eurospa cotton terry towel set size small ...,891
3,d4684dcdc759dd9cdf41504698d737d8,Key Features of SANTOSH ROYAL FASHION Cotton P...,Home Furnishing,858,key santosh royal fashion cotton print king si...,689
4,6325b6870c54cd47be6ebfbffa620ec7,Key Features of Jaipur Print Cotton Floral Kin...,Home Furnishing,1197,key jaipur print cotton floral king sized doub...,919


In [15]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uniq_id           1035 non-null   object
 1   description       1035 non-null   object
 2   category          1035 non-null   object
 3   _len_description  1035 non-null   int64 
 4   text              1035 non-null   object
 5   _len__text        1035 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 48.6+ KB


In [20]:
data['text']


'TfidfVectorizer()'

In [6]:
data = data[data['__len__text']<1300]
data


KeyError: '__len__text'

In [ ]:
# Unique categories extracted from the specified column.
l_cat = list(set(data['category']))

# Numerical labels corresponding to the unique categories.
y_cat_num = [ (1 - l_cat.index(data.iloc[i]['category'])) for i in range(len(data))]

# Create a dictionary to store the results of each model.
results = {}


### CountVectorizer

In [7]:
vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=1)
X = vectorizer.fit_transform(data['text'])

perplexities = [20, 30, 40]
learning_rates = [100, 200]

print("CountVectorizer : ")
print("-----------------")
params_ = {}
for perplexity in perplexities:
    for learning_rate in learning_rates:
      ari, tsne, labels = preprocess.perform_tsne_ari_kmeans(X, l_cat, y_cat_num,(2, perplexity, learning_rate,  2000, 42))
      params_[ari] = (perplexity, learning_rate)
best_parms = params_[max(params_.keys())]
logger.info("Best parameters for {} : perplexity = {} | learning_rate = {}".format(str(vectorizer)[:-2], best_parms[0], best_parms[1]))
ari, tsne, labels = preprocess.perform_tsne_ari_kmeans(X, l_cat, y_cat_num,(2, best_parms[0], best_parms[1],  2000, 42))
preprocess.visualize_tsne(tsne, labels, l_cat, y_cat_num)
logger.info(" {} --> Adjusted Rand Index : {}".format(str(vectorizer)[:-2], ari))
results[str(vectorizer)[:-2]] = max(params_.keys())



ValueError: np.nan is an invalid document, expected byte or unicode string.

#### TfidfVectorizer

In [11]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=1)
X = vectorizer.fit_transform(data['text'])

perplexities = [20, 30, 40]
learning_rates = [100, 200]

print("-----------------")
params_ = {}
for perplexity in perplexities:
    for learning_rate in learning_rates:
      ari, tsne, labels = preprocess.perform_tsne_ari_kmeans(X, l_cat, y_cat_num,(2, perplexity, learning_rate,  2000, 42))
      params_[ari] = (perplexity, learning_rate)
best_parms = params_[max(params_.keys())]
logger.info("Best parameters for {} : perplexity = {} | learning_rate = {}".format(str(vectorizer)[:-2], best_parms[0], best_parms[1]))
ari, tsne, labels = preprocess.perform_tsne_ari_kmeans(X, l_cat, y_cat_num,(2, best_parms[0], best_parms[1],  2000, 42))
preprocess.visualize_tsne(tsne, labels, l_cat, y_cat_num)
logger.info(" {} --> Adjusted Rand Index : {}".format(str(vectorizer), ari))
results[str(vectorizer)] = max(params_.keys())


ValueError: np.nan is an invalid document, expected byte or unicode string.

### Word2Vec

In [9]:
w2v_size=300
w2v_window=5
w2v_min_count=1
w2v_epochs=100
maxlen = 24 # adapt to length of sentences
sentences = data['text'].to_list()
sentences = [gensim.utils.simple_preprocess(text) for text in sentences]


TypeError: decoding to str: need a bytes-like object, float found

In [10]:
# Création et entraînement du modèle Word2Vec

print("Build & train Word2Vec model ...")
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")


Build & train Word2Vec model ...


TypeError: 'float' object is not iterable

In [ ]:
# Préparation des sentences (tokenization)

print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 
                                                   
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)


In [ ]:
# Création de la matrice d'embedding

print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0
    
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]
            
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))


In [ ]:
# Création du modèle

input=Input(shape=(len(x_sentences),maxlen),dtype='float64')
word_input=Input(shape=(maxlen,),dtype='float64')  
word_embedding=Embedding(input_dim=vocab_size,
                         output_dim=w2v_size,
                         weights = [embedding_matrix],
                         input_length=maxlen)(word_input)
word_vec=GlobalAveragePooling1D()(word_embedding)  
embed_model = Model([word_input],word_vec)

embed_model.summary()


In [ ]:
embeddings = embed_model.predict(x_sentences)
embeddings.shape


In [ ]:
perplexities = [20, 30, 40]
learning_rates = [100, 200]

print("-----------------")
params_ = {}
for perplexity in perplexities:
    for learning_rate in learning_rates:
      ari, tsne, labels = preprocess.perform_tsne_ari_kmeans(embeddings, l_cat, y_cat_num,(2, perplexity, learning_rate,  2000, 42))
      params_[ari] = (perplexity, learning_rate)
best_parms = params_[max(params_.keys())]
logger.info("Best parameters for {} : perplexity = {} | learning_rate = {}".format(str(vectorizer)[:-2], best_parms[0], best_parms[1]))
ari, tsne, labels = preprocess.perform_tsne_ari_kmeans(embeddings, l_cat, y_cat_num,(2, best_parms[0], best_parms[1],  2000, 42))
preprocess.visualize_tsne(tsne, labels, l_cat, y_cat_num)
logger.info(" {} --> Adjusted Rand Index : {}".format(str(vectorizer), ari))
results[str(vectorizer)] = max(params_.keys())


In [ ]:
# # Bert
# import os
# import transformers
# from transformers import *

# os.environ["TF_KERAS"]='1'
